In [22]:
import json
from datetime import datetime, date

In [23]:
j = open('user-tweets.json')

In [24]:
tweetsElon = []
for line in j:
    tweetsElon.append(json.loads(line))

In [25]:
from pymongo import MongoClient 
myclient = MongoClient("mongodb://localhost:27017/") 
db = myclient["DataMan"]
Collection = db["ElonTweets"]

In [26]:
for row in tweetsElon:
    row.pop("UserName", None)
    row.pop("LinkToTweet", None)
    row.pop("TweetEmbedCode", None)
    datetimeStr = datetime.strptime(row["CreatedAt"], '%B %d, %Y at %I:%M%p')
    row["Date"] = datetimeStr
    row["Day"] = datetime(datetimeStr.year, datetimeStr.month, datetimeStr.day)
    row.pop("CreatedAt", None)

In [27]:
#keep only tweets after startDay
startDay = datetime(2020, 6, 1)
tweetsElon = [x for x in tweetsElon if (startDay - x["Date"]).days < 0]
tweetsElon = [x for x in tweetsElon if "doge" in x["Text"] or "Doge" in x["Text"]]

In [28]:
if isinstance(tweetsElon, list):
    Collection.insert_many(tweetsElon)  
else:
    Collection.insert_one(tweetsElon)
myclient.close()